In [1]:
# CNN常见的术语
# channel，通道数，一般的RGB图像即为3通道数图像，在进行卷积时，通过设置卷积核的数量，可以调整输出的通道数
# 通常卷积核会对一张图片的多个通道进行卷积操作，之后进行图像信息融合（一般为求和），得到一个通道输出的图像
# 一般地，如果是3通道的图片，需要采用3个卷积核分别进行操作，计算完之后进行融合。
# 如果想要得到m个通道的输出图像，那么就需要m个批次的卷积核组，每个组卷积核数量为原图像的通道数
# maxpooling部分，主要用来实现对图像的下采样，通过保留局部最突出的信息，对图像进行压缩
# 一般指定maxpooling层的大小为2，意味着在2×2的单元格中找出最大值，如一个4×4大小的图像通过maxpooling操作之后，尺寸会变化成2×2
# patch指由一张图片多个通道组成的批量数据
# stride步长，也就是每次卷积核移动的grid数
# padding指在原图片四周边框所添加的像素，一般为0，或者取邻域的最大值，通常当padding=1时，使用3×3的卷积核卷积后图像的大小不改变
# batch，在pytorch当中，输入数据的格式一般为（batch，channe，width，height），batch指输入数据的批次，单位为batch_size

In [2]:
import torch.nn as nn
import torch
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
import torch.nn.functional as F
import torch.optim as optim
import os
import gzip
import numpy as np

In [3]:
class MNISTDataset(Dataset):
    def __init__(self, root, train=True, transform=None):
        self.root = root
        self.train = train
        self.transform = transform

        if self.train:
            images_file = 'train-images-idx3-ubyte.gz'
            labels_file = 'train-labels-idx1-ubyte.gz'
        else:
            images_file = 't10k-images-idx3-ubyte.gz'
            labels_file = 't10k-labels-idx1-ubyte.gz'

        self.images_path = os.path.join(root, images_file)
        self.labels_path = os.path.join(root, labels_file)

        self.images, self.labels = self.load_data()

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = int(self.labels[idx])

        if self.transform:
            image = self.transform(image)

        return image, label

    def load_data(self):
        with gzip.open(self.images_path, 'rb') as f_images:
            images = np.frombuffer(f_images.read(), dtype=np.uint8, offset=16).reshape(-1, 28, 28)

        with gzip.open(self.labels_path, 'rb') as f_labels:
            labels = np.frombuffer(f_labels.read(), dtype=np.uint8, offset=8)

        return images, labels

In [4]:
batch_size = 64
transform = transforms.Compose([
transforms.ToTensor(),
transforms.Normalize((0.1307, ), (0.3081, ))
])

In [5]:
train_dataset = MNISTDataset(root='./mnist_dataset/', train=True, transform=transform)
train_loader =  DataLoader(train_dataset,shuffle=True,batch_size=batch_size)

test_dataset = MNISTDataset(root='./mnist_dataset/', train=False, transform=transform)
test_loader =  DataLoader(test_dataset,shuffle=False,batch_size=batch_size)
print(test_dataset.__len__())
print(train_dataset.__len__())

10000
60000


In [6]:
class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model,self).__init__()
        self.conv1 = torch.nn.Conv2d(1,10,kernel_size=5)
        self.conv2 = torch.nn.Conv2d(10,20,kernel_size=5)
        self.pool = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(320,10)
        
    def forward(self,x):
        batch_size = x.shape[0]
        x = F.relu(self.pool(self.conv1(x)))
        x = F.relu(self.pool(self.conv2(x)))
        x = x.view(batch_size,-1)
        x = self.fc(x)
        return x
    

model = CNN_Model()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
        

CNN_Model(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=320, out_features=10, bias=True)
)

In [7]:
# construct loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [8]:
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader,0):
        inputs, target = data
        inputs, target = inputs.to(device), target.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs,target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if batch_idx%300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 300))
            running_loss = 0.0

        

In [9]:
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('accuracy on test set: %d %% ' % (100*correct/total))
    return correct/total



In [ ]:
import matplotlib.pyplot as plt

if __name__ == '__main__':
    epoch_list = []
    acc_list = []
    
    for epoch in range(10):
        train(epoch)
        acc = test()
        epoch_list.append(epoch)
        acc_list.append(acc)
    
    plt.plot(epoch_list,acc_list)
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.show()



D:\application\Anaconda\envs\pytorch-gpu\lib\site-packages\torchvision\transforms\functional.py:152: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:212.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


[1,   300] loss: 0.555
[1,   600] loss: 0.180
[1,   900] loss: 0.136
accuracy on test set: 96 % 
[2,   300] loss: 0.107
[2,   600] loss: 0.102
[2,   900] loss: 0.086
accuracy on test set: 97 % 
[3,   300] loss: 0.078
[3,   600] loss: 0.078
[3,   900] loss: 0.069
accuracy on test set: 98 % 
[4,   300] loss: 0.063
[4,   600] loss: 0.063
[4,   900] loss: 0.065
accuracy on test set: 98 % 
[5,   300] loss: 0.050
[5,   600] loss: 0.054
[5,   900] loss: 0.059
accuracy on test set: 98 % 
[6,   300] loss: 0.048
[6,   600] loss: 0.051
[6,   900] loss: 0.052
accuracy on test set: 98 % 
[7,   300] loss: 0.044
[7,   600] loss: 0.048
[7,   900] loss: 0.042
accuracy on test set: 98 % 
[8,   300] loss: 0.045
[8,   600] loss: 0.041
[8,   900] loss: 0.039
accuracy on test set: 98 % 
[9,   300] loss: 0.037
[9,   600] loss: 0.040
[9,   900] loss: 0.039
accuracy on test set: 98 % 
[10,   300] loss: 0.037
[10,   600] loss: 0.036
[10,   900] loss: 0.036
